<a href="https://colab.research.google.com/github/TXH2020/MainRepo/blob/main/Academics/Image%20Processing/AttProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install --upgrade -q gspread
!pip install face_recognition
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
import numpy as np
import gspread
import pandas as pd
from google.colab import auth
from google.colab import drive
from google.auth import default
import pickle
import os
import gdown

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
worksheet_name='Database Of Face Photos (Responses)'
worksheet = gc.open(worksheet_name).sheet1
rows = worksheet.get_all_values()
df=pd.DataFrame.from_records(rows,columns=rows[0],nrows=1)
df1=df.drop(0)
df1.reset_index
df1.set_index(np.arange(0,len(rows)-1),inplace=True)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import face_recognition
from PIL import Image
# Load the jpg files into numpy arrays
def encoding(df,face_encoding):
  known_image=[0 for i in range(len(face_encoding)+df.shape[0])]
  for i in range(len(face_encoding),len(face_encoding)+df.shape[0]):
    print(df['NAME'][i]+".jpg")
    known_image[i]=face_recognition.load_image_file(df['NAME'][i]+".jpg")

# Get the face encodings for each face in each image file
# Since there could be more than one face in each image, it returns a list of encodings.
# But since I know each image only has one face, I only care about the first encoding in each image, so I grab index 0.
  for i in range(len(face_encoding),len(face_encoding)+df.shape[0]):
    try:
      print("Extracting face encoding for photo:",i)
      face_encoding.append(face_recognition.face_encodings(known_image[i],model='cnn')[0])
  #unknown_face_encoding = face_recognition.face_encodings(unknown_image,model='cnn')
    except IndexError:
      im1=Image.open(df['NAME'][i]+".jpg")
      im2=im1.copy().rotate(270)
      im2.save('check.jpg')
      error_image=face_recognition.load_image_file("check.jpg")
      try:
        face_encoding.append(face_recognition.face_encodings(error_image,model='cnn')[0])
      except:
        im1=Image.open('check.jpg')
        im2=im1.copy().rotate(180)
        im2.save('check1.jpg')
        error_image1=face_recognition.load_image_file("check1.jpg")
        face_encoding.append(face_recognition.face_encodings(error_image1,model='cnn')[0])
  store_data(face_encoding)


In [44]:
def download_photos(df,l1):
 for i in range(len(l1),len(l1)+df.shape[0]):
  print("Downloading photo:",i)
  a=df['Your Photo'][i]
  file_id=a[a.find('=')+1:]
  url="https://drive.google.com/uc?id={}".format(file_id)
  gdown.download(url, df['NAME'][i]+".jpg", quiet=True)
 encoding(df,l1)

In [45]:
def store_data(face_encoding):
  %cd drive/MyDrive
  with open(worksheet_name+'.pkl','wb') as p:
    pickle.dump(face_encoding,p)
  %cd /content

In [46]:
if(os.path.isfile('drive/MyDrive/'+worksheet_name+'.pkl')==False):
  download_photos(df1,[])

%cd drive/MyDrive
with open(worksheet_name+'.pkl','rb') as p:
    face_encoding=pickle.load(p)
%cd /content

if(len(face_encoding)<df1.shape[0]):
  download_photos(df1.iloc[len(face_encoding):],face_encoding)

/content/drive/MyDrive
/content


In [47]:
ui=face_recognition.load_image_file('drive/MyDrive/group.jpg')
unknown_face_encoding=face_recognition.face_encodings(ui,model='cnn')

In [48]:
d={}
l1=[0 for i in range(len(face_encoding))]
for i in range(len(unknown_face_encoding)):
  result=face_recognition.face_distance(face_encoding, unknown_face_encoding[i])
  val=min(result)
  if(val<0.6):
    place=list(result).index(val)
    if(l1[place]!=1):
          d[place]=[val,i]
          l1[place]=1
    else:
      if(val<d[place][0]):
        fem=face_encoding.copy()
        fem.pop(place)
        im=d[place][1]
        resultm=face_recognition.face_distance(fem, unknown_face_encoding[im])
        valm=min(resultm)
        if(valm<0.6):
          placem=list(resultm).index(valm)
          if(l1[placem]!=1):
            d[place]=[valm,im]
            l1[place]=1

for i in range(len(l1)):
  if(l1[i]==1):
    print(df1['NAME'][i], " is Present")
  else:
    print(df1['NAME'][i], " is Absent")

ARNAV A RAJESH   is Present
Vinay B Sahani   is Absent
MADHUMITA   is Present
Swastik Puri  is Present
Tejas Hegde   is Present
Sanskar R Gondkar  is Present
ARKODEEP KOLEY  is Present
Chakradhar  is Present
ANKESH MOHAN NAIK   is Absent
Kushal R   is Present
Ninaad P S  is Absent
GANESH SHEKHAR NAIK  is Present
SUBHAJJI SUHAS   is Present
Shiya Singh  is Absent
SKANDA S KUMAR   is Present
Bharath M B  is Present
Abhinav S  is Absent
Manas Kulshrestha  is Absent
SHAWN DANIEL RODRIGUES  is Absent
R Jayanth Jadhav  is Present
Supreeth KG  is Present
